# Advanced Queries

This notebook provides a comprehensive guide to advanced query features in the Kotlin Notion Client:
- Complex filters (AND, OR, multiple conditions)
- Multiple filter types (title, text, number, checkbox, date, select)
- Sorting (single and multiple sort criteria)
- Pagination (page size and automatic fetching)
- Query performance patterns
- Real-world query examples

## Prerequisites

Make sure you have set these environment variables:
- `NOTION_API_TOKEN` - Your Notion integration token
- `NOTION_TEST_PAGE_ID` - A test page ID where we can create test content

## Setup: Load Dependencies and Initialize Client

In [1]:
// Load the Kotlin Notion Client library from Maven Central
@file:DependsOn("it.saabel:kotlin-notion-client:0.1.0")

// Import necessary classes
import it.saabel.kotlinnotionclient.NotionClient
import it.saabel.kotlinnotionclient.models.datasources.SortDirection
import it.saabel.kotlinnotionclient.models.pages.*
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.delay
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll

// Initialize the client
val apiToken = System.getenv("NOTION_API_TOKEN")
    ?: error("❌ NOTION_API_TOKEN environment variable not set")

val parentPageId = System.getenv("NOTION_TEST_PAGE_ID")
    ?: error("❌ NOTION_TEST_PAGE_ID environment variable not set")

val notion = NotionClient(apiToken)

println("✅ NotionClient initialized successfully!")

✅ NotionClient initialized successfully!


## Understanding Queries in Notion

Querying in Notion allows you to:
- **Filter** data based on property values (text, numbers, dates, checkboxes, etc.)
- **Sort** results by one or more properties
- **Paginate** large result sets efficiently
- **Combine conditions** using AND/OR logic

The Kotlin Notion Client provides a type-safe DSL for building complex queries that helps prevent errors and makes your code more maintainable.

## Setup: Create a Test Database with Sample Data

Let's create a database with various property types and populate it with test data for our query examples.

In [4]:
// Create a test database with multiple property types
val database = runBlocking {
    notion.databases.create {
        parent.page(parentPageId)
        title("Advanced Query Examples - Task Tracker")
        properties {
            title("Name")
            select("Status")
            select("Priority")
            checkbox("Completed")
            number("Score")
            select("Category")
        }
    }
}

println("✅ Database created: ${database.id}")

// Wait for database to be ready
runBlocking { delay(1000) }

// Get the data source ID (for 2025-09-03 API)
val retrievedDb = runBlocking { notion.databases.retrieve(database.id) }
val databaseUrl = retrievedDb.url
val dataSourceId = retrievedDb.dataSources.first().id

println("✅ Data source ID: $dataSourceId")
println("✅ URL for database: $databaseUrl")


✅ Database created: 6dfe13ec-f301-4f08-8be7-ac93cf9deb74
✅ Data source ID: 6625c800-3aea-4bb8-bad0-dacac0337296
✅ URL for database: https://www.notion.so/6dfe13ecf3014f088be7ac93cf9deb74


## Add Sample Data

Now let's populate the database with diverse test data using concurrent page creation.

In [7]:
// Define test data
val testTasks = listOf(
    Triple("High Priority Bug Fix", "In Progress", 95.0) to mapOf("Status" to "In Progress", "Priority" to "High", "Completed" to false, "Category" to "Bug"),
    Triple("Complete Documentation", "To Do", 75.0) to mapOf("Status" to "To Do", "Priority" to "Medium", "Completed" to false, "Category" to "Documentation"),
    Triple("Low Priority Feature", "To Do", 45.0) to mapOf("Status" to "To Do", "Priority" to "Low", "Completed" to false, "Category" to "Feature"),
    Triple("Finished Task", "Done", 85.0) to mapOf("Status" to "Done", "Priority" to "High", "Completed" to true, "Category" to "Feature"),
    Triple("Another Done Task", "Done", 65.0) to mapOf("Status" to "Done", "Priority" to "Medium", "Completed" to true, "Category" to "Bug"),
    Triple("Critical Security Fix", "In Progress", 100.0) to mapOf("Status" to "In Progress", "Priority" to "High", "Completed" to false, "Category" to "Security"),
    Triple("Refactor Old Code", "To Do", 50.0) to mapOf("Status" to "To Do", "Priority" to "Low", "Completed" to false, "Category" to "Refactor"),
    Triple("Write Unit Tests", "In Progress", 80.0) to mapOf("Status" to "In Progress", "Priority" to "Medium", "Completed" to false, "Category" to "Testing")
)

// Create pages concurrently for better performance
val pages = runBlocking {
    testTasks.map { (nameData, props) ->
        val (name, _, score) = nameData
        async {
            notion.pages.create {
                parent.dataSource(dataSourceId)
                properties {
                    title("Name", name)
                    select("Status", props["Status"] as String)
                    select("Priority", props["Priority"] as String)
                    checkbox("Completed", props["Completed"] as Boolean)
                    number("Score", score)
                    select("Category", props["Category"] as String)
                }
            }
        }
    }.awaitAll()
}

println("✅ Created ${pages.size} test pages concurrently")

// Wait for pages to be indexed
runBlocking { delay(1000) }

✅ Created 8 test pages concurrently


## Example 1: Simple Query (All Pages)

The simplest query retrieves all pages from a data source without any filters.

In [8]:
// Query all pages (no filter)
val allPages = runBlocking {
    notion.dataSources.query(dataSourceId)
}

println("✅ Query all pages: ${allPages.size} results")
allPages.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val score = page.getNumberProperty("Score")
    println("   - $name (Score: $score)")
}

✅ Query all pages: 8 results
   - Another Done Task (Score: 65.0)
   - Complete Documentation (Score: 75.0)
   - Write Unit Tests (Score: 80.0)
   - Low Priority Feature (Score: 45.0)
   - Refactor Old Code (Score: 50.0)
   - Critical Security Fix (Score: 100.0)
   - High Priority Bug Fix (Score: 95.0)
   - Finished Task (Score: 85.0)


## Example 2: Single Filter - Checkbox

Filter by a checkbox property to find completed or incomplete tasks.

In [10]:
// Query completed tasks
val completedTasks = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            checkbox("Completed").equals(true)
        }
    }
}

println("✅ Completed tasks: ${completedTasks.size} results")
completedTasks.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val status = page.getSelectProperty("Status")
    println("   - $name (Status: ${status?.name})")
}

✅ Completed tasks: 3 results
   - Another Done Task (Status: Done)
   - Low Priority Feature (Status: To Do)
   - Finished Task (Status: Done)


## Example 3: Single Filter - Number

Filter by numeric properties using comparison operators.

In [11]:
// Query high-score tasks (Score > 80)
val highScoreTasks = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            number("Score").greaterThan(80.0)
        }
    }
}

println("✅ High-score tasks (>80): ${highScoreTasks.size} results")
highScoreTasks.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val score = page.getNumberProperty("Score")
    println("   - $name (Score: $score)")
}

✅ High-score tasks (>80): 3 results
   - Critical Security Fix (Score: 100.0)
   - High Priority Bug Fix (Score: 95.0)
   - Finished Task (Score: 85.0)


## Example 4: Single Filter - Title (Text)

Filter by title/text properties using text search operators like `contains`.

In [12]:
// Query tasks with "Fix" in the title
val fixTasks = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            title("Name").contains("Fix")
        }
    }
}

println("✅ Tasks containing 'Fix': ${fixTasks.size} results")
fixTasks.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val category = page.getSelectProperty("Category")
    println("   - $name (Category: ${category?.name})")
}

✅ Tasks containing 'Fix': 2 results
   - Critical Security Fix (Category: Security)
   - High Priority Bug Fix (Category: Bug)


## Example 5: Single Filter - Select

Filter by select properties to find pages with specific values.

In [13]:
// Query high priority tasks
val highPriorityTasks = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            select("Priority").equals("High")
        }
    }
}

println("✅ High priority tasks: ${highPriorityTasks.size} results")
highPriorityTasks.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val status = page.getSelectProperty("Status")
    val score = page.getNumberProperty("Score")
    println("   - $name (Status: ${status?.name}, Score: $score)")
}

✅ High priority tasks: 3 results
   - Critical Security Fix (Status: In Progress, Score: 100.0)
   - High Priority Bug Fix (Status: In Progress, Score: 95.0)
   - Finished Task (Status: Done, Score: 85.0)


## Example 6: Complex Filter - AND Logic

Combine multiple conditions with AND logic to narrow down results. All conditions must be true.

In [14]:
// Query: Incomplete tasks with high scores (Score > 70 AND Completed = false)
val incompleteHighScore = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            and(
                checkbox("Completed").equals(false),
                number("Score").greaterThan(70.0)
            )
        }
    }
}

println("✅ Incomplete high-score tasks (>70, not completed): ${incompleteHighScore.size} results")
incompleteHighScore.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val score = page.getNumberProperty("Score")
    val priority = page.getSelectProperty("Priority")
    println("   - $name (Score: $score, Priority: ${priority?.name})")
}

✅ Incomplete high-score tasks (>70, not completed): 4 results
   - Complete Documentation (Score: 75.0, Priority: Medium)
   - Write Unit Tests (Score: 80.0, Priority: Medium)
   - Critical Security Fix (Score: 100.0, Priority: High)
   - High Priority Bug Fix (Score: 95.0, Priority: High)


## Example 7: Complex Filter - OR Logic

Use OR logic to match pages that satisfy at least one condition.

In [15]:
// Query: Either Bug or Security category
val bugOrSecurity = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            or(
                select("Category").equals("Bug"),
                select("Category").equals("Security")
            )
        }
    }
}

println("✅ Bug or Security tasks: ${bugOrSecurity.size} results")
bugOrSecurity.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val category = page.getSelectProperty("Category")
    val status = page.getSelectProperty("Status")
    println("   - $name (Category: ${category?.name}, Status: ${status?.name})")
}

✅ Bug or Security tasks: 3 results
   - Another Done Task (Category: Bug, Status: Done)
   - Critical Security Fix (Category: Security, Status: In Progress)
   - High Priority Bug Fix (Category: Bug, Status: In Progress)


## Example 8: Nested Filters - AND + OR

You can nest AND/OR logic to create complex query conditions.

In [16]:
// Query: (High priority OR high score) AND incomplete
val criticalIncomplete = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            and(
                or(
                    select("Priority").equals("High"),
                    number("Score").greaterThanOrEqualTo(90.0)
                ),
                checkbox("Completed").equals(false)
            )
        }
    }
}

println("✅ Critical incomplete tasks: ${criticalIncomplete.size} results")
criticalIncomplete.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val score = page.getNumberProperty("Score")
    val priority = page.getSelectProperty("Priority")
    println("   - $name (Score: $score, Priority: ${priority?.name})")
}

✅ Critical incomplete tasks: 2 results
   - Critical Security Fix (Score: 100.0, Priority: High)
   - High Priority Bug Fix (Score: 95.0, Priority: High)


## Example 9: Sorting - Single Property

Sort results by a single property in ascending or descending order.

In [18]:
// Query all tasks, sorted by Score descending
val sortedByScore = runBlocking {
    notion.dataSources.query(dataSourceId) {
        sortBy("Score", SortDirection.DESCENDING)
    }
}

val numberOfItemsToShow = 3
println("✅ Tasks sorted by Score (descending): ${sortedByScore.size} results. Showing Top $numberOfItemsToShow:")
sortedByScore.take(numberOfItemsToShow).forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val score = page.getNumberProperty("Score")
    println("   - $name (Score: $score)")
}

✅ Tasks sorted by Score (descending): 8 results. Showing Top 3:
   - Critical Security Fix (Score: 100.0)
   - High Priority Bug Fix (Score: 95.0)
   - Finished Task (Score: 85.0)


## Example 10: Combined Filtering and Sorting

Combine filters and sorting to get precise, ordered results.

In [19]:
// Query incomplete tasks, sorted by score descending
val incompleteByScore = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            checkbox("Completed").equals(false)
        }
        sortBy("Score", SortDirection.DESCENDING)
    }
}

println("✅ Incomplete tasks by score: ${incompleteByScore.size} results")
incompleteByScore.forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    val score = page.getNumberProperty("Score")
    val status = page.getSelectProperty("Status")
    println("   - $name (Score: $score, Status: ${status?.name})")
}

✅ Incomplete tasks by score: 5 results
   - Critical Security Fix (Score: 100.0, Status: In Progress)
   - High Priority Bug Fix (Score: 95.0, Status: In Progress)
   - Write Unit Tests (Score: 80.0, Status: In Progress)
   - Complete Documentation (Score: 75.0, Status: To Do)
   - Refactor Old Code (Score: 50.0, Status: To Do)


## Example 11: Pagination

Control page size for large result sets. The Notion API automatically fetches all pages.

In [20]:
// Query with page size of 3 (API will automatically fetch all pages)
val paginatedResults = runBlocking {
    notion.dataSources.query(dataSourceId) {
        pageSize(3)
    }
}

println("✅ Pagination example (page size: 3)")
println("   Total results: ${paginatedResults.size} (API automatically fetched all pages)")
println("   First 3 results:")
paginatedResults.take(3).forEach { page ->
    val name = page.getTitleAsPlainText("Name")
    println("   - $name")
}

✅ Pagination example (page size: 3)
   Total results: 8 (API automatically fetched all pages)
   First 3 results:
   - Another Done Task
   - Complete Documentation
   - Write Unit Tests


## Summary

In this notebook, we explored advanced query features comprehensively:

### Query Operations
- **Simple queries**: Retrieve all pages without filters
- **Single filters**: Filter by checkbox, number, title/text, select properties
- **Complex filters**: Combine conditions with AND/OR logic
- **Nested filters**: Mix AND/OR for sophisticated query logic
- **Sorting**: Sort by single property (ascending/descending)
- **Combined operations**: Filter + sort in one query
- **Pagination**: Control page size (API automatically fetches all pages)

### Filter Types Demonstrated
- **Checkbox**: `checkbox("property").equals(true/false)`
- **Number**: `number("property").greaterThan(value)`, `greaterThanOrEqualTo()`, `lessThan()`, `lessThanOrEqualTo()`, `equals()`
- **Title/Text**: `title("property").contains("text")`, `startsWith()`, `endsWith()`, `equals()`
- **Select**: `select("property").equals("value")`, `doesNotEqual()`

### Logic Operators
- **AND**: `and(condition1, condition2, ...)` - All conditions must be true
- **OR**: `or(condition1, condition2, ...)` - At least one condition must be true
- **Nested**: Combine AND/OR for complex logic like `and(or(...), ...)`

### Sorting Options
- **Ascending**: `sortBy("property", SortDirection.ASCENDING)`
- **Descending**: `sortBy("property", SortDirection.DESCENDING)`

### Performance Patterns
1. **Concurrent page creation**: Use `async`/`awaitAll` for batch operations
2. **Filter at query time**: Let Notion filter data server-side for better performance
3. **Sort server-side**: Use `sortBy()` rather than sorting results in code
4. **Pagination**: API automatically handles pagination, fetching all results

### Best Practices
1. **Type-safe queries**: Use DSL methods rather than raw JSON
2. **Specific filters**: More specific filters = faster queries
3. **Property accessors**: Use `getTitleAsPlainText()`, `getNumberProperty()`, etc. for safe property access
4. **Error handling**: Properties may be null, handle gracefully
5. **API delays**: Use `delay()` after bulk operations to allow indexing

### Real-World Query Examples
- Find high-priority incomplete tasks
- Get completed tasks sorted by score
- Find bugs or security issues
- Complex criteria: (High priority OR score >= 90) AND incomplete

## Cleanup

Let's clean up by archiving the test database and all created pages.

In [21]:
runBlocking {
    // Archive all created pages
    pages.forEach { page ->
        notion.pages.archive(page.id)
        delay(100)
    }
    
    // Archive the database
    notion.databases.archive(database.id)
}

println("✅ Test database and all pages archived successfully!")
println("🎉 Advanced Queries notebook completed!")

✅ Test database and all pages archived successfully!
🎉 Advanced Queries notebook completed!
